In [ ]:
stu_df.dropna(subset=['Financial Stress'], inplace=True)

In [ ]:
stu_train_data_features = stu_train_data_features.applymap(lambda x: 1 if x is True else (0 if x is False else x))
stu_train_data_features = stu_train_data_features.apply(pd.to_numeric, errors='coerce')
stu_train_data_features = stu_train_data_features.astype({col: 'int' for col in stu_train_data_features.select_dtypes('bool').columns})




In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

X_tensor = torch.tensor(stu_train_data_features.values, dtype=torch.float32)
y_tensor = torch.tensor(stu_df_label.values, dtype=torch.long)


# Create a dataset
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset (80% training, 20% testing)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.optim as optim

class EnhancedNN(nn.Module):
    def __init__(self, input_size):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)  # Output layer for binary classification
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # Dropout layer for regularization

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.fc4(x)
        return x

input_size = X_tensor.shape[1]  # Number of features
model = EnhancedNN(input_size)

In [ ]:
criterion = nn.CrossEntropyLoss()  # Good for binary/multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
import matplotlib.pyplot as plt

num_epochs = 200  # or however many epochs you want to train for
train_losses = []  # To store the training loss for each epoch

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Plotting the training loss
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to compute gradients for evaluation
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")
